In [1]:

    import pygsheets
    import pandas as pd
    import numpy as np
    import datetime as dt
    import time
    import filepath
    import pandas as pd
    import glob
    import os
    import numpy as np
    import filepaths
    import send_email 
    import pygsheets
    import infrastructure
    import datetime
    import math
    import datetime
    from datetime import timedelta  
    import re
    import warnings
    import statistics
    from datetime import date
    from calendar import monthrange
    from dateutil.relativedelta import relativedelta
    import xlsxwriter
    import send_email
    from colorama import Fore, Style
    import matplotlib.pyplot as plt
    import seaborn as sns
    import filepath 
    import time
    import datetime as dt



    warnings.filterwarnings('ignore', '.*apply Jump Page Clickpyxl.*', )
    warnings.filterwarnings('ignore', '.*Specify dtype option on import.*', )
    warnings.filterwarnings('ignore', '.*empty string*', )
    warnings.filterwarnings('ignore', '.*invalid value encountered in double_scalars*', )
    warnings.filterwarnings("ignore", message="divide by zero encountered")
    warnings.filterwarnings("ignore", message="Can't merge single cell")


    pd.options.mode.chained_assignment = None  # default='warn'
    pd.set_option('display.max_columns',50)
    pd.set_option('display.max_rows',50)
    # https://docs.google.com/document/d/1-k1sEDbp_PXPW4esXftDhvqp3zOm7Y0FPsLmImY3kvs/edit

    from os import listdir
    from os.path import isfile, join


    print('Loading Lexi Data..')
    # import offers. 
    offer_sms = infrastructure.get_smartsheet('offers_sms')
    offer_sms = offer_sms[offer_sms['Hitpath Offer ID'].isna() == False]
    # customize part to merge email report
    lexi =  pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
    lexi_copy = lexi.copy() 
    lexi['Date'] = pd.to_datetime(lexi['Date'])
    lexi = lexi.rename(columns = {"Hitpath_Offer_ID": 'Hitpath Offer ID'})
    lexi = lexi.loc[~lexi['Send Strategy'].isin(['Null','Opt In',np.nan])]
    lexi = lexi[lexi['Date']>= '2022-11-01']
    mamba = infrastructure.get_mamba()
    mamba['Affiliate ID'] = mamba['Dataset'].str.split('_',expand = True)[2].astype(int)
    
    #
    # import publisher information 
    publisher  = infrastructure.get_publisher()
    publisher = publisher.drop_duplicates(subset='PUBID', keep='last')
    publisher['PUBID'] = publisher['PUBID'].astype(str).str.split('.',expand = True)[0]
    current_active_pubid= publisher[publisher['INTERNAL STATUS']== 'ACTIVE']['PUBID'].unique().tolist()
    lexi['Affiliate_Id'] = lexi['Affiliate_Id'].astype(str).str.split('.',expand = True)[0]
    lexi = lexi[lexi['Affiliate_Id'].isin(current_active_pubid)]
    lexi = lexi.merge(publisher[['PUBID','DP.DS or DP.sV']], how = 'left', left_on ='Affiliate_Id', right_on = 'PUBID')
    lexi['PUBID1'] = lexi['PUBID'].astype(str).str.split('.',expand = True)[0]
    lexi['DP&Pub'] = lexi['DP.DS or DP.sV']+'_'+ lexi['PUBID1']
    lexi['SC_DP&Pub'] = ''
    lexi.loc[(lexi['DP&Pub'].isnull() == False) & (lexi['Shortcode Name'].isnull() == False), 'SC_DP&Pub'] = lexi['Shortcode Name'] + '_' + lexi['DP&Pub']
    # replace contact lost with Optout , replace opens with Jump Page Clicks
    # email team dont have Jump Page Clicks but we have 
    #########
    lexi['eCPM'] = lexi['Revenue']*1000 / lexi['Delivered']
    lexi['CTR'] = lexi['Clicks']*1000 / lexi['Delivered']
    lexi['eCPM Ratio'] = (lexi['Revenue'])/((lexi['Revenue'])-lexi['Opportunity Cost'])
    lexi['CTR Normalized']=(lexi['CTR']-lexi['CTR'].min())/(lexi['CTR'].max()-lexi['CTR'].min())
    lexi['eCPM Normalized']=(lexi['eCPM']-lexi['eCPM'].min())/(lexi['eCPM'].max()-lexi['eCPM'].min())
    lexi['CTR50'] = lexi['CTR Normalized'] + lexi['eCPM Normalized']
    lexi['Profit'] = lexi['Revenue'].fillna(0) - lexi['Cost'].fillna(0)
    lexi_clean = lexi.copy()
    
    ## import segment data 
    segment = pd.read_csv(filepath.output_folder+'SMS_SC_SS_Segments.csv')
    segment['Date'] = pd.to_datetime(segment['Date'])
    segment_max_date = segment['Date'].max()
    mamba = infrastructure.get_mamba()
    upcoming_mamba = mamba[(mamba['Date'] > segment_max_date) ]
    upcoming_segment = upcoming_mamba['Segment '].str.strip().unique().tolist()
    mamba_dic_df = infrastructure.get_mamba_directory()
    current_active_segment = mamba_dic_df['Segment'].str.split('\n').tolist()
    current_active_segment = [item for sublist in current_active_segment for item in sublist if item]
  
    #current_active_segment = mamba.loc[mamba['Date'] == pd.to_datetime(date.today()),'Segment '].unique().tolist()
    #secondary_segment =  mamba.loc[(mamba['Date'] == pd.to_datetime(date.today())) & (mamba['Segment '].str.contains('TMO|VZN',na = False)),'Segment '].unique().tolist()

    
    print('Lexi Loading Complete!')
    #import post data 
    post = pd.read_csv(filepath.output_folder+'SMSposts.csv')
    post['Dates'] = pd.to_datetime(post['Dates'])
    # import sms report data 
    smsreport = pd.read_csv(filepath.output_folder+'SMS_SC_SS_Report.csv')
    smsreport['Date'] = pd.to_datetime(smsreport['Date'])
    current_month = lexi['Date'].max().month
    current_year = lexi['Date'].max().year
    
    # sort DP&Pub and SC_Dp& pUB by last 6 digit 
    def get_last_six_digits(item):
        return item[-6:]
    def get_dppub(item):
        return item.split("_")[1]
    
    current_month_df = lexi[(lexi['Date'].dt.month==current_month) & (lexi['Date'].dt.year==current_year)]
    current_month_dp_pubs = current_month_df['DP&Pub'].unique()
    current_month_dp_pubs = current_month_dp_pubs[~pd.isnull(current_month_dp_pubs)]
    current_month_dp_pubs = sorted(current_month_dp_pubs, key=get_last_six_digits)
    current_month_sc_pubs = current_month_df['SC_DP&Pub'].unique()
    current_month_sc_pubs = current_month_sc_pubs[~pd.isnull(current_month_sc_pubs) | (current_month_sc_pubs!='')]
    current_month_sc_pubs = sorted(current_month_sc_pubs, key=get_last_six_digits)
    current_active_segment = sorted(current_active_segment, key=get_dppub)
    
    lexi['Date'] = pd.to_datetime(lexi['Date'])
    current_date = dt.date.today()  # Get the current date
    max_date = current_date - dt.timedelta(days=1)
    
    today_df = lexi[(lexi['Date'].dt.date== max_date) ]
    today_df_dp_pubs= today_df['DP&Pub'].unique()
    today_df_sc_pubs= today_df['SC_DP&Pub'].unique()
    today_df_sc= today_df['Shortcode Name'].unique()
    today_df_sc = today_df_sc[~pd.isnull(today_df_sc)]
    today_df_sc_pubs = [x for x in today_df_sc_pubs if x != '' and pd.isnull(x) == False]
    today_df_dp_pubs = today_df_dp_pubs[~pd.isnull(today_df_dp_pubs)]
    today_df_dp_pubs = sorted(today_df_dp_pubs, key=get_last_six_digits)
    today_df_sc_pubs = sorted(today_df_sc_pubs)
    post['Dates'] = post['Dates'].dt.date
    smsreport['Date'] = smsreport['Date'].dt.date
    # rename to Optout
    smsreport = smsreport.rename(columns = {'Opt-out':'Optout'})
    
    last_7_days = max_date - dt.timedelta(days = 7)
    last_7d_df = lexi[(lexi['Date'].dt.date< max_date ) & (lexi['Date'].dt.date>=  last_7_days) ]
    post['active'] = 0
    post.loc[post['Pub Id'].isin(publisher[publisher['PUBID']!= '']['PUBID'].astype('float').unique().tolist()),'active'] = 1

    today_df_post = post[(post['Dates']== max_date) ]
    post_df_pubs= today_df_post.sort_values(['active'],ascending= False)['Pub Id'].unique()
    today_df_optout = smsreport[(smsreport['Date']== max_date) ]
    today_df_segment = segment[(segment['Date']== segment_max_date) ]
    last_7_days_segment = segment_max_date - dt.timedelta(days = 7)
    
    last_7d_df_post = post[(post['Dates']< max_date ) & (post['Dates']>=  last_7_days) ]
    last_7d_df_optout = smsreport[(smsreport['Date']< max_date ) & (smsreport['Date']>=  last_7_days) ]
    last_7d_df_segment = segment[(segment['Date']< segment_max_date ) & (segment['Date']>=  last_7_days_segment) ]
    
    days_passed = len(current_month_df['Date'].unique())
    total_days_in_month = monthrange(current_year,current_month)[1]

    last_month = (lexi['Date'].max() + relativedelta(months=-1)).month
    last_month_year = (lexi['Date'].max() + relativedelta(months=-1)).year
    last_month_df = lexi[(lexi['Date'].dt.month==last_month) & (lexi['Date'].dt.year==last_month_year)]

    three_month = (lexi['Date'].max() + relativedelta(months=-3)).month
    three_month_year = (lexi['Date'].max() + relativedelta(months=-3)).year
    three_month_df = lexi[(lexi['Date'].dt.month==three_month) & (lexi['Date'].dt.year==three_month_year)]
    
    six_month = (lexi['Date'].max() + relativedelta(months=-6)).month
    six_month_year = (lexi['Date'].max() + relativedelta(months=-6)).year
    six_month_df = lexi[(lexi['Date'].dt.month==six_month) & (lexi['Date'].dt.year==six_month_year)]

    period = lexi['Date'].dt.to_period("M")
    g = lexi.groupby([period,'DP&Pub'])
    g_sc = lexi.groupby([period,'SC_DP&Pub'])

    simple_metrics = {'Revenue':'sum','Cost':'sum','Profit':'sum','Delivered':'sum'}
    agg_metrics = {'eCPM':'Revenue','CTR':'Clicks','Jump Page Click Rate':'Jump Page Clicks','Optout Rate':'Optout','Margin':'Profit'}
    

    grouped = g.agg({'Revenue':'sum','Cost':'sum','Profit':'sum','Delivered':'sum','Jump Page Clicks':'sum','Clicks':'sum','Optout':'sum'})

    g_consec = lexi.groupby(['DP&Pub',period])
    grouped_consec = g_consec.agg({'Revenue':'sum','Cost': 'sum','Profit':'sum','Delivered':'sum','Jump Page Clicks':'sum','Clicks':'sum','Optout':'sum'})
    grouped_consec = grouped_consec.reset_index()

    three_month_df_grouped = grouped.loc['{}-{}'.format(three_month_year,three_month):'{}-{}'.format(current_year,current_month)].reset_index()
    six_month_df_grouped = grouped.loc['{}-{}'.format(six_month_year,six_month):'{}-{}'.format(current_year,current_month)].reset_index()
    last_month_df_grouped = grouped.loc['{}-{}'.format(last_month_year,last_month):'{}-{}'.format(current_year,current_month)].reset_index()




    grouped_sc = g_sc.agg({'Revenue':'sum','Cost':'sum','Profit':'sum','Delivered':'sum','Jump Page Clicks':'sum','Clicks':'sum','Optout':'sum'})

    three_month_df_grouped_sc = grouped_sc.loc['{}-{}'.format(three_month_year,three_month):'{}-{}'.format(current_year,current_month)].reset_index()
    six_month_df_grouped_sc = grouped_sc.loc['{}-{}'.format(six_month_year,six_month):'{}-{}'.format(current_year,current_month)].reset_index()
    last_month_df_grouped_sc = grouped_sc.loc['{}-{}'.format(last_month_year,last_month):'{}-{}'.format(current_year,current_month)].reset_index()



    

Loading Lexi Data..


D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')
D:\Users\lilig\Desktop\github\SMS_Report\Daily_Reporting\infrastructure.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  schedule2['Date']=pd.to_datetime(schedule2[0],errors='coerce')


Lexi Loading Complete!


In [2]:
zero_ot_email_body = "* Here are Pipeline Testing Offers with $0 Revenue, please check whether there's tracking issue:\n"
zero_ot_email_body += "\tTableau Link: https://tableau.rxmg.app/#/views/RXMG-SMSReports/0DropsTable?:iid=1"+"\n\n"
last_7d_df['Hitpath Offer ID'] = last_7d_df['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
temp = last_7d_df.loc[last_7d_df['Send Strategy'] == 'PT',].groupby('Hitpath Offer ID')['Revenue'].sum()
zero_drops = temp[temp == 0].reset_index(name = 'Revenue')
if len(zero_drops) != 0 : 
    
    offer_sms['Hitpath Offer ID'] = offer_sms['Hitpath Offer ID'].astype(str).str.split('.',expand = True)[0]
    zero_drops = zero_drops.merge(offer_sms[['Hitpath Offer ID','Scheduling Name']], how = 'left',left_on = 'Hitpath Offer ID',right_on = 'Hitpath Offer ID')
    for i in zero_drops['Hitpath Offer ID'].unique(): 
        scheduling_name = zero_drops[zero_drops['Hitpath Offer ID']==i]['Scheduling Name'].values[0]
        zero_ot_email_body += '\tOffer '+scheduling_name+":\n\tHere are drops with $0 Revenue:\n"
        
        izero_drops = last_7d_df.loc[last_7d_df['Hitpath Offer ID'] == i, ] 
        for j in range(0,len(izero_drops)): 
            temp_df = izero_drops.iloc[j,:]
            date_temp_df =temp_df['Date'].strftime("%Y-%m-%d")
            dp_pub_temp_df = temp_df['DP&Pub']
            zero_ot_email_body += '\t  '+date_temp_df+'\t'+dp_pub_temp_df+"\n"
else: 
    zero_ot_email_body += "\tThere's no $0 Revenue pipeline testing offers in the last 7 days.\n"
          

In [3]:
push = lexi_copy[lexi_copy['Revenue Source'] =='Push']
# get last 7 days push data
push['Date'] = pd.to_datetime(push['Date'])
last_30_days = max_date - dt.timedelta(days = 30)

push7 = push[(push['Date'].dt.date<= max_date ) & (push['Date'].dt.date>  last_7_days) ]
push_rev_7_gpby = push7.groupby('Date')['Revenue'].sum().round(2)
push_rev_7 = push_rev_7_gpby.reset_index(name = 'Revenue')
avg_push_rev_7 = push_rev_7['Revenue'].mean() 
push_email_body = "* Here are push revenue in the last 7 days:\n"
# each line of push_rev_7_gpby in push_email_body 
for i in range(0,len(push_rev_7_gpby)):
    temp_df = push_rev_7.iloc[i,:]
    date_temp_df =temp_df['Date'].strftime("%Y-%m-%d")
    rev_temp_df = temp_df['Revenue']
    push_email_body += '\t'+date_temp_df+'\t'+str(rev_temp_df)+"\n"
push_rev_7_below_avg = push_rev_7[push_rev_7['Revenue']<avg_push_rev_7*0.8]
if len(push_rev_7_below_avg) != 0 : 
    push_email_body  += "Please pay attention to the following date as the push revenue is -20% below Average Daily Push Revenue in the last 7 days: " + "\n"
    for i in range(0,len(push_rev_7_below_avg)):
        temp_df = push_rev_7_below_avg.iloc[i,:]
        date_temp_df =temp_df['Date'].strftime("%Y-%m-%d")
        rev_temp_df = temp_df['Revenue']
        push_email_body += '\t'+date_temp_df + "\n"


In [4]:

    filename = filepath.output_folder+'Account Feedback - {}.xlsx'.format(date.today().strftime("%m_%d_%Y"))
    workbook = xlsxwriter.Workbook(filename)
    optout_sheet =  workbook.add_worksheet('Daily Optout')
    post_sheet = workbook.add_worksheet('Daily Raw Post')
    segment_sheet = workbook.add_worksheet('Daily Segment')
    acct_worksheet = workbook.add_worksheet('Account Metrics by Shortcode')
    acct_last_month_sheet = workbook.add_worksheet('SC Revenue Decliners Last Month')
    worksheet = workbook.add_worksheet('Account Metrics')
    last_month_sheet = workbook.add_worksheet('Revenue Decliners - Last Month')
    header_fmt = workbook.add_format({'bold': 1,'align': 'center','valign': 'vcenter','text_wrap': True})
    percent_fmt = workbook.add_format({'num_format': '0.00%'})
    optout_alert = workbook.add_format({'num_format': '0.00%','bg_color':   '#F5B7B1'})
    post_uptrend_alert = workbook.add_format({'num_format': '0.00%','bg_color':   '#f6e877'})
    post_alert = workbook.add_format({'num_format': '#,###','bg_color':   '#F5B7B1'})
    post_rgl_fmt = workbook.add_format({'num_format': '#,###'})
    dollar_fmt = workbook.add_format({'num_format': '$#,###.##'})
    no_fmt = workbook.add_format({'num_format': '#,###.##'})
    # dollar_fmt = workbook.add_format({'num_format': '0.00%'})
    worksheet.set_column(0,8,25)
    acct_worksheet.set_column(0,8,25)
    acct_last_month_sheet.set_column(0,8,25)
    last_month_sheet.set_column(0,8,25)
    segment_sheet.set_column(0,7,25)
    optout_sheet.set_column(0,8,25)
    post_sheet.set_column(0,5,30)
    # worksheet.set_column(1,1,15)

    serious_downtrend_pub = {}
    # three_month_dict = {}
    # six_month_dict = {}
    # last_year_month_dict = {}
    last_month_dict = {}

    last_month_count = -1
    
    delivered_email_body = "* Shortcode Delivered Volumn\n"
    delivered_email_body += "\t(The data source is SMS Report, please note there's some discrepancy between Tableau)\n"
    delivered_email_body += "\tTableau Link: https://tableau.rxmg.app/views/Performance_Dashboard/DailyDeliveredovertimebyShortcode-sum?:showAppBanner=false&:display_count=n&:showVizHome=n&:origin=viz_share_link"+"\n\n"
    
    
    ## Calculate optout table
    serious_account_optout = {}
    optout_alert_email_body = "* Here're the account with over 2% optout rate:\n"
    optout_alert_email_body += "\tTableau Link: https://tableau.rxmg.app/views/Performance_Dashboard_16873822397360/Optout?:showAppBanner=false&:display_count=n&:showVizHome=n&:origin=viz_share_link"+"\n\n"
    
    # segment email body 
    segment_email_body = "* Here's the production segment under 500 volume:\n"
    segment_email_body +="\tTableau Link: https://tableau.rxmg.app/#/views/Segments/SegmentSizeDash?:iid=1"+"\n\n"
    small_segment = {}
    # post alert
    serious_account_post = {}
    post_downtrend_email_body = "Here're the accounts with 20%+ decrease in the raw post: \n\n "
    post_downtrend_email_body += "\tDash Link: https://dash.rxmg.tech/analysis/dashboard"+"\n\n"
    post_uptrend_email_body = "Here're the accounts with 20%+ increase in the raw post: \n\n "
    post_alert_email_body ="* Here're the accounts with less than 3k raw posts: \n\n"
    subcribed_downtrend_email_body = "Here're the accounts with 20%+ decrease in Subscribed compared to daily average last 7 days: \n\n "
    subcribed_uptrend_email_body = "Here're the accounts with 20%+ increase in Subscribed compared to daily average last 7 days: \n\n "
    subscribed_email_body ="* Here're the accounts with 0 subscribed: \n\n"
    for index, sc_dp_pub in enumerate(post_df_pubs):
            # Metric today
            today = today_df_post['Dates'].values[0]
            today_sc_dp_pub_df = today_df_post[today_df_post['Pub Id']==sc_dp_pub]
            pubname = today_sc_dp_pub_df['Pub Name'].values[0]
            dp_pub =  str(sc_dp_pub)[:6] + ' '+ pubname
            today_post =  today_sc_dp_pub_df['Raw Posts'].sum() 
            today_subscribed =    today_sc_dp_pub_df['Subscribed'].sum() 

            last_7d_sc_dp_pub_df = last_7d_df_post[last_7d_df_post['Pub Id']==sc_dp_pub]
            last_7d_post =  last_7d_sc_dp_pub_df['Raw Posts'].sum() / 7 
            last_7d_subscribed =  last_7d_sc_dp_pub_df['Subscribed'].sum()  / 7 
            subscribed_pct = today_subscribed / today_post

            if today_post <3000:
                serious_account_post[dp_pub] = today_post 
                post_alert_email_body += "\n\t"+dp_pub +": \n\t\tRaw Posts Number: "+str(today_post)+"\n"
                if (today_post >0) & (last_7d_post ==0): 
                    post_alert_email_body += "\t\t\t"+ dp_pub +" start to have raw post from "+str(today)
            if today_subscribed == 0 :
                subscribed_email_body += "\n\t"+dp_pub +": \n\t\tSubscribed Number: 0 \n"
            

            ## % Difference from last 7 days. 
            last_7d_diff = (today_post - last_7d_post) / last_7d_post
            last_7d_diff_subscribed = (today_subscribed - last_7d_subscribed) / last_7d_subscribed
            if (today_post >0) & (last_7d_diff <-0.2): 
                post_downtrend_email_body += "\n\t"+ dp_pub + ": \n\t\tRaw Posts Number: "+str(today_post)+"\n\t\tRaw Posts decreased by "+str(round(last_7d_diff*100,2))+"% compared to daily average post last 7 days\n"        
            if (today_post >0) & (last_7d_diff >0.2): 
                post_uptrend_email_body += "\n\t"+ dp_pub + ": \n\t\tRaw Posts Number: "+str(today_post)+"\n\t\tRaw Posts increase by "+str(round(last_7d_diff*100,2))+"% compared to daily average post last 7 days\n"        
            if (today_subscribed >0) & (last_7d_diff_subscribed <-0.2): 
                subcribed_downtrend_email_body += "\n\t"+ dp_pub + ": \n\t\tSubscribed Number: "+str(today_subscribed)+"\n\t\tSubscribed decreased by "+str(round(last_7d_diff_subscribed*100,2))+"%\n"        
            if (today_subscribed >0) & (last_7d_diff_subscribed >0.2): 
                subcribed_uptrend_email_body += "\n\t"+ dp_pub + ": \n\t\tSubscribed Number: "+str(today_subscribed)+"\n\t\tSubscribed increase by "+str(round(last_7d_diff_subscribed*100,2))+"%\n"        

            try: 
                post_sheet.write('A{}'.format(index+2), sc_dp_pub, header_fmt)
            except: 
                pass
            try: 
                post_sheet.write('B{}'.format(index+2), pubname, header_fmt)
            except: 
                pass
            try:
                post_sheet.write('C{}'.format(index+2),today_post,post_rgl_fmt)           
                if today_post <3000:
                    post_sheet.write('C{}'.format(index+2),today_post,post_alert)
            except:
                pass
            try:
                post_sheet.write('D{}'.format(index+2),last_7d_post,post_rgl_fmt)
            except:
                pass
            try:
                post_sheet.write('E{}'.format(index+2),last_7d_diff,percent_fmt)
                if (last_7d_diff <= -0.2) & (today_post >0): 
                    post_sheet.write('E{}'.format(index+2),last_7d_diff,optout_alert)
                if (last_7d_diff >= 0.2) & (today_post >0): 
                    post_sheet.write('E{}'.format(index+2),last_7d_diff,post_uptrend_alert)
            except:
                pass
            try:
                post_sheet.write('F{}'.format(index+2),subscribed_pct,percent_fmt)
            except:
                pass
            try:
                post_sheet.write('G{}'.format(index+2),today_subscribed,post_rgl_fmt)
                if today_subscribed == 0:
                    post_sheet.write('G{}'.format(index+2),today_subscribed,post_alert)
            except:
                pass
            try:
                post_sheet.write('H{}'.format(index+2),last_7d_subscribed,post_rgl_fmt)
            except:
                pass
            try:
                post_sheet.write('I{}'.format(index+2),last_7d_diff_subscribed,percent_fmt)
            except:
                pass

    post_sheet.write('A1','DP&Pub',header_fmt)
    post_sheet.write('B1', "Publisher Name",header_fmt)
    post_sheet.write('C1', "Yesterday's Post",header_fmt)
    post_sheet.write('D1',"Daily Average Post Last 7 Days",header_fmt)
    post_sheet.write('E1','% Difference\nfrom Last 7 Days - Post',header_fmt)
    post_sheet.write('F1', "Yesterday's Subscribed % ",header_fmt)
    post_sheet.write('G1', "Yesterday's Subscribed",header_fmt)
    post_sheet.write('H1',"Daily Average Subscribed Last 7 Days",header_fmt)
    post_sheet.write('I1','% Difference\nfrom Last 7 Days - Subscribed',header_fmt)

    for index, sc_dp_pub in enumerate(current_active_segment):
            # Metric today
            today = today_df_segment['Date'].values[0]
            today_sc_dp_pub_df = today_df_segment[today_df_segment['name']==sc_dp_pub]
            pubname = sc_dp_pub.split("_")[1]
            dp_pub =  str(sc_dp_pub)[:6] + ' '+ pubname
            today_segment =  today_sc_dp_pub_df['count'].sum()    

            last_7d_sc_dp_pub_df = last_7d_df_segment[last_7d_df_segment['name']==sc_dp_pub]
            last_7d_segment =  last_7d_sc_dp_pub_df['count'].sum() / len(last_7d_sc_dp_pub_df['Date'].unique() )
            last_7d_df_avg_revenue = last_7d_df.loc[last_7d_df['Segments'] ==sc_dp_pub ]['Revenue'].sum()/7
            if today_segment <500:
                small_segment[sc_dp_pub] = today_segment 
                segment_email_body += "\n\t"+dp_pub +": \n\t\t"+sc_dp_pub+" segment size: "+str(today_segment)+"\n"
            if (today_segment >=500) and (sc_dp_pub not in upcoming_segment): 
                segment_email_body += "\n\t"+dp_pub +": \n\t\t"+sc_dp_pub+" segment size: "+str(today_segment)+"\n"
                segment_email_body += "\t\t\t"+"Please start sending production drops" +"\n"
            
            ## % Difference from last 7 days. 
            last_7d_diff = (today_segment - last_7d_segment) / last_7d_segment
            
            try: 
                segment_sheet.write('A{}'.format(index+2), pubname, header_fmt)
            except: 
                pass
            try: 
                segment_sheet.write('B{}'.format(index+2), sc_dp_pub, header_fmt)
            except: 
                pass
            try:
                segment_sheet.write('C{}'.format(index+2),today_segment,post_rgl_fmt)           
                if today_segment <500:
                    segment_sheet.write('C{}'.format(index+2),today_segment,post_alert)
            except:
                pass
            try:
                segment_sheet.write('D{}'.format(index+2),last_7d_segment,post_rgl_fmt)
            except:
                pass
            try:
                segment_sheet.write('E{}'.format(index+2),last_7d_diff,percent_fmt)
            except:
                pass
            try:
                segment_sheet.write('F{}'.format(index+2),last_7d_df_avg_revenue,dollar_fmt)
            except:
                pass

    segment_sheet.write('A1','DP&Pub',header_fmt)
    segment_sheet.write('B1', "Segment",header_fmt)
    segment_sheet.write('C1', "Today's Segment Size",header_fmt)
    segment_sheet.write('D1',"Daily Average Segment Size Last 7 Days",header_fmt)
    segment_sheet.write('E1','% Difference\nfrom Last 7 Days',header_fmt)
    segment_sheet.write('F1','Avg Daily Revenue Last 7 Days',header_fmt)
            
    for index, sc_dp_pub in enumerate(today_df_sc):
            # Metric today

            today_sc_dp_pub_df = today_df_optout[today_df_optout['Shortcode Name']==sc_dp_pub]
            total_delivered = today_sc_dp_pub_df['Delivered'].sum()   
            today_optout =  today_sc_dp_pub_df['Optout'].sum() /   today_sc_dp_pub_df['Delivered'].sum() 
            
            delivered_email_body += "\tDelivered Volume for Shortcode " +sc_dp_pub+": "+  str(round(total_delivered,0))+"\n"
            if today_optout > 0.02:
                serious_account_optout[sc_dp_pub] = today_optout 
                optout_alert_email_body += "\tShortcode " +sc_dp_pub+": "+ str(round(today_optout*100,2)) + '%\n'
                
            
            last_7d_sc_dp_pub_df = last_7d_df_optout[last_7d_df_optout['Shortcode Name']==sc_dp_pub]
               
            last_7d_optout =  last_7d_sc_dp_pub_df['Optout'].sum() /   last_7d_sc_dp_pub_df['Delivered'].sum()  
            """ 
            if sc_dp_pub == 'SVT':
                today_sc_dp_pub_df = today_df[today_df['Shortcode Name']==sc_dp_pub] 
                last_7d_sc_dp_pub_df = last_7d_df[last_7d_df['Shortcode Name']==sc_dp_pub]
                total_delivered = today_sc_dp_pub_df['Delivered'].sum()   
                today_optout =today_sc_dp_pub_df['Optout'].sum() /   today_sc_dp_pub_df['Delivered'].sum() 
                delivered_email_body += "\tDelivered Volume for Longcode " +sc_dp_pub+": "+  str(round(total_delivered,0))+"\n"
                last_7d_optout =  last_7d_sc_dp_pub_df['Optout'].sum() /   last_7d_sc_dp_pub_df['Delivered'].sum()  
            """
            # AR optout 
            
            today_ar_optout=  np.nan
            today_ar_optout_delivered = np.nan
            # non-AR optout 
            
            today_p_optout=  np.nan
            today_p_optout_delivered = np.nan
            
            
            ## % Difference from last 7 days. 
            last_7d_diff = (today_optout - last_7d_optout) / last_7d_optout
            
            try: 
                optout_sheet.write('A{}'.format(index+2), sc_dp_pub, header_fmt)
            except: 
                pass
            try:
                optout_sheet.write('B{}'.format(index+2),today_optout,percent_fmt)
                            
                if today_optout > 0.02:
                    optout_sheet.write('B{}'.format(index+2),today_optout,optout_alert)
                
            except:
                pass
            try:
                optout_sheet.write('C{}'.format(index+2),total_delivered,post_rgl_fmt)
            except:
                pass            
            try:
                optout_sheet.write('D{}'.format(index+2),last_7d_optout,percent_fmt)
            except:
                pass
            try:
                optout_sheet.write('E{}'.format(index+2),last_7d_diff,percent_fmt)
            except:
                pass
            try:
                optout_sheet.write('F{}'.format(index+2),today_ar_optout,percent_fmt)
                if today_ar_optout > 0.02:
                    optout_sheet.write('F{}'.format(index+2),today_ar_optout,optout_alert)
            except:
                pass
            try:
                optout_sheet.write('G{}'.format(index+2),today_ar_optout_delivered,post_rgl_fmt)
            except:
                pass
            try:
                optout_sheet.write('H{}'.format(index+2),today_p_optout,percent_fmt)
                              
                if today_p_optout > 0.02:
                    optout_sheet.write('H{}'.format(index+2),today_p_optout,optout_alert)
            except:
                pass
            try:
                optout_sheet.write('I{}'.format(index+2),today_p_optout_delivered,post_rgl_fmt)
            except:
                pass
 
    for index, sc_dp_pub in enumerate(today_df_sc_pubs):
            # Metric today
            no_sc = len(today_df_sc)
            today_sc_dp_pub_df = today_df[today_df['SC_DP&Pub']==sc_dp_pub]
            today_optout_delivered =  today_sc_dp_pub_df['Delivered'].sum()   
            today_optout =  today_sc_dp_pub_df['Optout'].sum() /   today_sc_dp_pub_df['Delivered'].sum()   

                
            
            last_7d_sc_dp_pub_df = last_7d_df[last_7d_df['SC_DP&Pub']==sc_dp_pub]
            last_7d_optout =  last_7d_sc_dp_pub_df['Optout'].sum() /   last_7d_sc_dp_pub_df['Delivered'].sum() 
            
            
            
            # AR optout 
            today_sc_dp_pub_df_ar = today_sc_dp_pub_df.loc[today_sc_dp_pub_df['Send Strategy'] == 'AR', ]
            today_ar_optout_delivered =  today_sc_dp_pub_df_ar['Delivered'].sum()
            today_ar_optout=  today_sc_dp_pub_df_ar['Optout'].sum() /   today_sc_dp_pub_df_ar['Delivered'].sum()  
            
            # non-AR optout 
            today_sc_dp_pub_df_p = today_sc_dp_pub_df.loc[today_sc_dp_pub_df['Send Strategy'].isin( ['P','CT']), ]
            today_p_optout_delivered =  today_sc_dp_pub_df_p['Delivered'].sum()
            today_p_optout=  today_sc_dp_pub_df_p['Optout'].sum() /   today_sc_dp_pub_df_p['Delivered'].sum()
            if today_optout > 0.02:
                serious_account_optout[sc_dp_pub] = today_optout 
                optout_alert_email_body += "\n\tAccount " +sc_dp_pub+": "+ str(round(today_optout*100,2)) + '%\n'
                if today_ar_optout > 0.02:
                    optout_alert_email_body += "\t\tMain reason: AR drops with " + str(round(today_ar_optout*100,2)) + '% optout rate\n'
                if today_p_optout > 0.02:
                    optout_alert_email_body += "\t\tMain reason: P drops with " + str(round(today_p_optout*100,2)) + '% optout rate\n'
            
            ## % Difference from last 7 days. 
            last_7d_diff = (today_optout - last_7d_optout) / last_7d_optout
            
            try: 
                optout_sheet.write('A{}'.format(index+2+no_sc), sc_dp_pub, header_fmt)
            except: 
                pass
            try:
                optout_sheet.write('B{}'.format(index+2+no_sc),today_optout,percent_fmt)
                            
                if today_optout > 0.02:
                    optout_sheet.write('B{}'.format(index+2+no_sc),today_optout,optout_alert)
                
            except:
                pass
            try:
                optout_sheet.write('C{}'.format(index+2+no_sc),today_optout_delivered,post_rgl_fmt)
            except:
                pass            
            try:
                optout_sheet.write('D{}'.format(index+2+no_sc),last_7d_optout,percent_fmt)
            except:
                pass
            try:
                optout_sheet.write('E{}'.format(index+2+no_sc),last_7d_diff,percent_fmt)
            except:
                pass
            try:
                optout_sheet.write('F{}'.format(index+2+no_sc),today_ar_optout,percent_fmt)
                if today_ar_optout > 0.02:
                    optout_sheet.write('F{}'.format(index+2+no_sc),today_ar_optout,optout_alert)
            except:
                pass
            try:
                optout_sheet.write('G{}'.format(index+2+no_sc),today_ar_optout_delivered,post_rgl_fmt)
            except:
                pass
            try:
                optout_sheet.write('H{}'.format(index+2+no_sc),today_p_optout,percent_fmt)
                              
                if today_p_optout > 0.02:
                    optout_sheet.write('H{}'.format(index+2+no_sc),today_p_optout,optout_alert)
            except:
                pass
            try:
                optout_sheet.write('I{}'.format(index+2+no_sc),today_p_optout_delivered,post_rgl_fmt)
            except:
                pass

    optout_sheet.write('A1','DP&Pub',header_fmt)
    optout_sheet.write('B1', "Yesterday's Optout Rate",header_fmt)
    optout_sheet.write('C1',"Yesterday's Delivered Volume",header_fmt)
    optout_sheet.write('D1',"Last 7 Days' Optout Rate",header_fmt)
    optout_sheet.write('E1',"% Difference\nfrom Last 7 Days",header_fmt)
    optout_sheet.write('F1',"Yesterday's AR Optout Rate",header_fmt)
    optout_sheet.write('G1',"Yesterday's AR Delivered Volume",header_fmt)
    optout_sheet.write('H1',"Yesterday's Production Optout Rate",header_fmt)
    optout_sheet.write('I1',"Yesterday's Production Delivered Volume",header_fmt)
   
    
    for index,dp_pub in enumerate(current_month_dp_pubs):
        write_last_month = False
        for simple_index,metric in enumerate(simple_metrics):

            # Metric Month to Date
            current_month_dp_pub_df = current_month_df[current_month_df['DP&Pub']==dp_pub]
            mtd = current_month_dp_pub_df[metric].agg(simple_metrics[metric])
            

            # Straight Line Pacing
            if (simple_metrics[metric]=='sum') | (simple_metrics[metric]=='count'):
                straight_line = mtd*total_days_in_month/days_passed
            else:
                straight_line = mtd


            # Last Month's Metric
            last_month = last_month_df[last_month_df['DP&Pub']==dp_pub][metric].agg(simple_metrics[metric])
            last_month_proj_diff = (straight_line - last_month) / last_month

    #         % Difference from 3 Month Max
            three_month_max = three_month_df_grouped[three_month_df_grouped['DP&Pub']==dp_pub][metric].max()
            three_month_max_diff = (straight_line - three_month_max) / three_month_max

            # % Difference from 12 Month Max
            six_month_max = six_month_df_grouped[six_month_df_grouped['DP&Pub']==dp_pub][metric].max()
            six_month_max_diff = (straight_line - six_month_max) / six_month_max

            # % Difference from Last Year
            six_month_dp_pub_df = six_month_df[six_month_df['DP&Pub']==dp_pub]
            six_month_performance = six_month_dp_pub_df[metric].agg(simple_metrics[metric])
            six_month_performance_diff = (straight_line - six_month_performance) / six_month_performance

            if (metric=='Revenue') | (metric=='Opportunity Cost') | (metric=='Cost') | (metric=='Profit'):
                form = dollar_fmt
            elif metric=='eCPM Ratio':
                form = workbook.add_format({'num_format': '0.00'})
            else:
                form = no_fmt

            try:
                worksheet.write('C{}'.format(index*9+simple_index+2),mtd,form)
            except:
                pass
            try:
                worksheet.write('D{}'.format(index*9+simple_index+2),straight_line,form)
            except:
                pass
            try:
                worksheet.write('E{}'.format(index*9+simple_index+2),last_month,form)
            except:
                pass
            try:
                worksheet.write('F{}'.format(index*9+simple_index+2),last_month_proj_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('G{}'.format(index*9+simple_index+2),three_month_max_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('H{}'.format(index*9+simple_index+2),six_month_max_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('I{}'.format(index*9+simple_index+2),six_month_performance_diff,percent_fmt)
            except:
                pass

    #         if (three_month_max_diff < -0.3) & (three_month_max_diff != float('-inf')) :
    #             if three_month_dict.get(dp_pub) is None:
    #                 three_month_dict[dp_pub] = {metric:three_month_max_diff}
    #             else:
    #                 three_month_dict[dp_pub][metric] =  three_month_max_diff

    #         if (six_month_max_diff < -0.5) & (six_month_max_diff != float('-inf')) :
    #             if six_month_dict.get(dp_pub) is None:
    #                 six_month_dict[dp_pub] = {metric:six_month_max_diff}
    #             else:
    #                 six_month_dict[dp_pub][metric] =  six_month_max_diff

    #         if (six_month_performance_diff < -0.2) & (six_month_performance_diff != float('-inf')) :
    #             if last_year_month_dict.get(dp_pub) is None:
    #                 last_year_month_dict[dp_pub] = {metric:six_month_performance_diff}
    #             else:
    #                 last_year_month_dict[dp_pub][metric] =  six_month_performance_diff

            if (last_month_proj_diff < 0) & (last_month_proj_diff != float('-inf')) :
                if last_month_dict.get(dp_pub) is None:
                    last_month_dict[dp_pub] = {metric:last_month_proj_diff}
                else:
                    last_month_dict[dp_pub][metric] =  last_month_proj_diff

            if (metric=='Revenue') & (last_month_proj_diff < 0):
                write_last_month = True #note this works because "Revenue" is the first metric
                last_month_count += 1

            if write_last_month:
                try:
                    last_month_sheet.write('C{}'.format((last_month_count)*9+simple_index+2),mtd,form)
                except:
                    pass
                try:
                    last_month_sheet.write('D{}'.format((last_month_count)*9+simple_index+2),straight_line,form)
                except:
                    pass
                try:
                    last_month_sheet.write('E{}'.format((last_month_count)*9+simple_index+2),last_month,form)
                except:
                    pass
                try:
                    last_month_sheet.write('F{}'.format((last_month_count)*9+simple_index+2),last_month_proj_diff,percent_fmt)
                except:
                    pass
                try:
                    last_month_sheet.write('G{}'.format((last_month_count)*9+simple_index+2),three_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    last_month_sheet.write('H{}'.format((last_month_count)*9+simple_index+2),six_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    last_month_sheet.write('I{}'.format((last_month_count)*9+simple_index+2),six_month_performance_diff,percent_fmt)
                except:
                    pass


    #         #Populate Consecutive DF

    #         decline_series = grouped_consec[metric].diff().lt(0)

    #         zero_series = decline_series.cumsum().where(decline_series, 0) #where no change in series, replace with a 0
    #         true_false_zero_series = zero_series.eq(0)
    #         new_col_name = 'Consecutive Declining {} Months'.format(metric)
    #         grouped_consec[new_col_name] = true_false_zero_series.groupby(true_false_zero_series.cumsum()).cumcount()





        for agg_index,metric in enumerate(agg_metrics):

            # Metric Month to Date
    #         current_month_dp_pub_df = current_month_df[current_month_df['DP&Pub']==dp_pub]
            agg_metric = agg_metrics[metric]
            mtd = current_month_dp_pub_df[agg_metric].agg('sum') / current_month_dp_pub_df['Delivered'].agg('sum')
            if metric == 'Margin':
                mtd = current_month_dp_pub_df[agg_metric].agg('sum') / current_month_dp_pub_df['Revenue'].agg('sum')

            # Straight Line Pacing
    #         if simple_metrics[metric]=='sum':
    #             straight_line = mtd*total_days_in_month/days_passed
    #         else:

            # Last Month's Metric
            last_month = last_month_df[last_month_df['DP&Pub']==dp_pub][agg_metric].sum() / last_month_df[last_month_df['DP&Pub']==dp_pub]['Delivered'].sum()
            if metric == 'Margin':
                last_month =last_month_df[last_month_df['DP&Pub']==dp_pub][agg_metric].sum() / last_month_df[last_month_df['DP&Pub']==dp_pub]['Revenue'].sum()

            if metric=='eCPM':
                mtd *= 1000
                last_month *= 1000
                form = dollar_fmt

                three_month_df_grouped[metric] = 1000*three_month_df_grouped[agg_metrics[metric]]/three_month_df_grouped['Delivered']
                six_month_df_grouped[metric] = 1000*six_month_df_grouped[agg_metrics[metric]]/six_month_df_grouped['Delivered']
                last_month_df_grouped[metric] = 1000*last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Delivered']
                grouped_consec[metric] = 1000*grouped_consec['Revenue']/grouped_consec['Delivered']
            elif metric == 'Margin':
                form = percent_fmt
                three_month_df_grouped[metric] = three_month_df_grouped[agg_metrics[metric]]/three_month_df_grouped['Revenue']
                six_month_df_grouped[metric] = six_month_df_grouped[agg_metrics[metric]]/six_month_df_grouped['Revenue']
                last_month_df_grouped[metric] = last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Revenue']
                grouped_consec[metric] = grouped_consec[agg_metrics[metric]]/grouped_consec['Revenue']
            else:
                form = percent_fmt

                three_month_df_grouped[metric] = three_month_df_grouped[agg_metrics[metric]]/three_month_df_grouped['Delivered']
                six_month_df_grouped[metric] = six_month_df_grouped[agg_metrics[metric]]/six_month_df_grouped['Delivered']
                last_month_df_grouped[metric] = last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Delivered']
                grouped_consec[metric] = grouped_consec[agg_metrics[metric]]/grouped_consec['Delivered']
               


            straight_line = mtd
            # % Difference from 3 Month Max
            three_month_max = three_month_df_grouped[three_month_df_grouped['DP&Pub']==dp_pub][metric].max()
            three_month_max_diff = (straight_line - three_month_max) / three_month_max

    #         % Difference from 12 Month Max
            six_month_max = six_month_df_grouped[six_month_df_grouped['DP&Pub']==dp_pub][metric].max()
            six_month_max_diff = (straight_line - six_month_max) / six_month_max

    #         % Difference from Last Year
            six_month_dp_pub_df = six_month_df[six_month_df['DP&Pub']==dp_pub]
            six_month_performance = six_month_dp_pub_df[agg_metric].sum() / six_month_dp_pub_df['Delivered'].sum()
            if metric == 'Margin':
                six_month_performance = six_month_dp_pub_df[agg_metric].sum() / six_month_dp_pub_df['Revenue'].sum()
            if metric=='eCPM':
                six_month_performance*=1000
            six_month_performance_diff = (straight_line - six_month_performance) / six_month_performance 

            # % Difference from Last Month
            last_month_diff = (straight_line - last_month) / last_month


    #         print(six_month_max_diff==three_month_max_diff)
    #         print(dp_pub,metric,mtd,straight_line,last_month,three_month_max_diff,six_month_max_diff)
            try:
                worksheet.write('C{}'.format(index*9+agg_index+6),mtd,form)
            except:
                pass
            try:
                worksheet.write('D{}'.format(index*9+agg_index+6),straight_line,form)
            except:
                pass
            try:
                worksheet.write('E{}'.format(index*9+agg_index+6),last_month,form)
            except:
                pass
            try:
                worksheet.write('F{}'.format(index*9+agg_index+6),last_month_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('G{}'.format(index*9+agg_index+6),three_month_max_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('H{}'.format(index*9+agg_index+6),six_month_max_diff,percent_fmt)
            except:
                pass
            try:
                worksheet.write('I{}'.format(index*9+agg_index+6),six_month_performance_diff,percent_fmt)
            except:
                pass

    #         if three_month_max_diff < -0.3:
    #             if three_month_dict.get(dp_pub) is None:
    #                 three_month_dict[dp_pub] = {metric:three_month_max_diff}
    #             else:
    #                 three_month_dict[dp_pub][metric] =  three_month_max_diff

    #         if six_month_max_diff < -0.5:
    #             if six_month_dict.get(dp_pub) is None:
    #                 six_month_dict[dp_pub] = {metric:six_month_max_diff}
    #             else:
    #                 six_month_dict[dp_pub][metric] =  six_month_max_diff

    #         if six_month_performance_diff < -0.2:
    #             if last_year_month_dict.get(dp_pub) is None:
    #                 last_year_month_dict[dp_pub] = {metric:six_month_performance_diff}
    #             else:
    #                 last_year_month_dict[dp_pub][metric] =  six_month_performance_diff


            if last_month_diff < -0.2:
                if last_month_dict.get(dp_pub) is None:
                    last_month_dict[dp_pub] = {metric:last_month_diff}
                else:
                    last_month_dict[dp_pub][metric] =  last_month_diff

            if write_last_month:
                try:
                    last_month_sheet.write('C{}'.format((last_month_count)*9+agg_index+6),mtd,form)
                except:
                    pass
                try:
                    last_month_sheet.write('D{}'.format((last_month_count)*9+agg_index+6),straight_line,form)
                except:
                    pass
                try:
                    last_month_sheet.write('E{}'.format((last_month_count)*9+agg_index+6),last_month,form)
                except:
                    pass
                try:
                    last_month_sheet.write('F{}'.format((last_month_count)*9+agg_index+6),last_month_diff,percent_fmt)
                except:
                    pass
                try:
                    last_month_sheet.write('G{}'.format((last_month_count)*9+agg_index+6),three_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    last_month_sheet.write('H{}'.format((last_month_count)*9+agg_index+6),six_month_max_diff,percent_fmt)
                except:
                    pass
                
                try:
                    last_month_sheet.write('I{}'.format((last_month_count)*9+agg_index+6),six_month_performance_diff,percent_fmt)
                except:
                    pass
                
    #         #Populate Consecutive DF
    #         decline_series = grouped_consec[metric].diff().lt(0)

    #         zero_series = decline_series.cumsum().where(decline_series, 0) #where no change in series, replace with a 0
    #         true_false_zero_series = zero_series.eq(0)
    #         new_col_name = 'Consecutive Declining {} Months'.format(metric)
    #         grouped_consec[new_col_name] = true_false_zero_series.groupby(true_false_zero_series.cumsum()).cumcount()
    #         value_before_decline = 'Starting {}'.format(metric)
    # #         date_before_decline = '{} Starting Date'.format(metric)
    #         grouped_consec[value_before_decline] = grouped_consec[new_col_name].apply(lambda x: lexi['Date'].max() + pd.DateOffset(months=-x))



        try: 
            worksheet.merge_range('A{}:A{}'.format(index*9+2,index*9+10), dp_pub, header_fmt)
            worksheet.write('B{}'.format(index*9+2),'Revenue',header_fmt)
            worksheet.write('B{}'.format(index*9+3),'Cost',header_fmt)
            worksheet.write('B{}'.format(index*9+4),'Profit',header_fmt)
            worksheet.write('B{}'.format(index*9+5),'Delivered',header_fmt)
            worksheet.write('B{}'.format(index*9+6),'eCPM',header_fmt)
            worksheet.write('B{}'.format(index*9+7),'CTR',header_fmt)
            worksheet.write('B{}'.format(index*9+8),'Jump Page Click Rate',header_fmt)
            worksheet.write('B{}'.format(index*9+9),'Optout Rate',header_fmt)
            worksheet.write('B{}'.format(index*9+10),'Margin',header_fmt)
        except: 
            pass

        if write_last_month:
            

            last_month_sheet.merge_range('A{}:A{}'.format((last_month_count)*9+2,last_month_count*9+10), dp_pub, header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+2),'Revenue',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+3),'Cost',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+4),'Profit',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+5),'Delivered',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+6),'eCPM',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+7),'CTR',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+8),'Jump Page Click Rate',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+9),'Optout Rate',header_fmt)
            last_month_sheet.write('B{}'.format(last_month_count*9+10),'Margin',header_fmt)



    worksheet.write('A1','DP&Pub',header_fmt)
    worksheet.write('B1','Metric',header_fmt)
    worksheet.write('C1','Month to Date',header_fmt)
    worksheet.write('D1','Straight Line Pacing',header_fmt)
    worksheet.write('E1','Last Month Performance',header_fmt)
    worksheet.write('F1','Percent Difference\nfrom Last Month',header_fmt)
    worksheet.write('G1','Percent Difference\nfrom 3 Month Max',header_fmt)
    worksheet.write('H1','Percent Difference\nfrom 6 Month Max',header_fmt)
    worksheet.write('I1','Percent Difference\nfrom the Month 6 Months ago',header_fmt)

    last_month_sheet.write('A1','DP&Pub',header_fmt)
    last_month_sheet.write('B1','Metric',header_fmt)
    last_month_sheet.write('C1','Month to Date',header_fmt)
    last_month_sheet.write('D1','Straight Line Pacing',header_fmt)
    last_month_sheet.write('E1','Last Month Performance',header_fmt)
    last_month_sheet.write('F1','Percent Difference\nfrom Last Month',header_fmt)
    last_month_sheet.write('G1','Percent Difference\nfrom 3 Month Max',header_fmt)
    last_month_sheet.write('H1','Percent Difference\nfrom 6 MontMax',header_fmt)
    last_month_sheet.write('I1','Percent Difference\nfrom the Month 6 Months ago',header_fmt)


    # three_month_sheet = workbook.add_worksheet('Accounts Declining - 3 Months')
    # three_month_sheet.write('A1','DP&Pub',header_fmt)
    # three_month_sheet.write('B1','Metric',header_fmt)
    # three_month_sheet.write('C1','Projected Percent Difference\nfrom 3 Month Max',header_fmt)
    # three_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (three_month_dict):
    #     for counter,met in enumerate(three_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         three_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         three_month_sheet.write('C{}'.format(row_index+1),three_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         three_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         three_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 


    # six_month_sheet = workbook.add_worksheet('Accounts Declining - 12 Months')
    # six_month_sheet.write('A1','DP&Pub',header_fmt)
    # six_month_sheet.write('B1','Metric',header_fmt)
    # six_month_sheet.write('C1','Projected Percent Difference\nfrom 12 Month Max',header_fmt)
    # six_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (six_month_dict):
    #     for counter,met in enumerate(six_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         six_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         six_month_sheet.write('C{}'.format(row_index+1),six_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         six_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         six_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 

    # last_year_month_sheet = workbook.add_worksheet('Accounts Declining - Last Year')
    # last_year_month_sheet.write('A1','DP&Pub',header_fmt)
    # last_year_month_sheet.write('B1','Metric',header_fmt)
    # last_year_month_sheet.write('C1','Projected Percent Difference\nfrom 12 Month Max',header_fmt)
    # last_year_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (last_year_month_dict):
    #     for counter,met in enumerate(last_year_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         last_year_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         last_year_month_sheet.write('C{}'.format(row_index+1),last_year_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         last_year_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         last_year_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 


    # last_month_sheet_any = workbook.add_worksheet('Accounts Declining - Last Month')
    # last_month_sheet_any.write('A1','DP&Pub',header_fmt)
    # last_month_sheet_any.write('B1','Metric',header_fmt)
    # last_month_sheet_any.write('C1','Projected Percent Difference\nfrom Last Month',header_fmt)
    # last_month_sheet_any.set_column(0,7,25)
    # row_index = 1
    # for key in (last_month_dict):
    #     for counter,met in enumerate(last_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         last_month_sheet_any.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         last_month_sheet_any.write('C{}'.format(row_index+1),last_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         last_month_sheet_any.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         last_month_sheet_any.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt)   




    #     worksheet.merge_range('A{}:A{}'.format(index*9+2,index*9+10), dp_pub, header_fmt)

    #     worksheet.write('B{}'.format(index*9+2),'Revenue',header_fmt)
    #     worksheet.write('B{}'.format(index*9+3),'Delivered',header_fmt)
    #     worksheet.write('B{}'.format(index*9+4),'Number of Drops',header_fmt)
    #     worksheet.write('B{}'.format(index*9+5),'Opportunity Cost',header_fmt)
    #     worksheet.write('B{}'.format(index*9+6),'eCPM Ratio',header_fmt)
    #     worksheet.write('B{}'.format(index*9+7),'eCPM',header_fmt)
    #     worksheet.write('B{}'.format(index*9+8),'CTR',header_fmt)
    #     worksheet.write('B{}'.format(index*9+9),'Jump Page Click Rate',header_fmt)
    #     worksheet.write('B{}'.format(index*9+10),'Optout Rate',header_fmt)

    ############################ Extra Tap for SMS due to Shortcode 
    
    last_month_dict = {}

    last_month_count = -1
    for index,dp_pub in enumerate(current_month_sc_pubs):
        write_last_month = False
        for simple_index,metric in enumerate(simple_metrics):

            # Metric Month to Date
            current_month_dp_pub_df = current_month_df[current_month_df['SC_DP&Pub']==dp_pub]
            mtd = current_month_dp_pub_df[metric].agg(simple_metrics[metric])
            

            # Straight Line Pacing
            if (simple_metrics[metric]=='sum') | (simple_metrics[metric]=='count'):
                straight_line = mtd*total_days_in_month/days_passed
            else:
                straight_line = mtd


            # Last Month's Metric
            last_month = last_month_df[last_month_df['SC_DP&Pub']==dp_pub][metric].agg(simple_metrics[metric])
            last_month_proj_diff = (straight_line - last_month) / last_month

    #         % Difference from 3 Month Max
            three_month_max = three_month_df_grouped_sc[three_month_df_grouped_sc['SC_DP&Pub']==dp_pub][metric].max()
            three_month_max_diff = (straight_line - three_month_max) / three_month_max

            # % Difference from 12 Month Max
            six_month_max = six_month_df_grouped_sc[six_month_df_grouped_sc['SC_DP&Pub']==dp_pub][metric].max()
            six_month_max_diff = (straight_line - six_month_max) / six_month_max

            # % Difference from Last Year
            six_month_dp_pub_df = six_month_df[six_month_df['SC_DP&Pub']==dp_pub]
            six_month_performance = six_month_dp_pub_df[metric].agg(simple_metrics[metric])
            six_month_performance_diff = (straight_line - six_month_performance) / six_month_performance

            if (metric=='Revenue') | (metric=='Opportunity Cost') | (metric=='Cost') | (metric=='Profit'):
                form = dollar_fmt
            elif metric=='eCPM Ratio':
                form = workbook.add_format({'num_format': '0.00'})
            else:
                form = no_fmt

            try:
                acct_worksheet.write('C{}'.format(index*9+simple_index+2),mtd,form)
            except:
                pass
            try:
                acct_worksheet.write('D{}'.format(index*9+simple_index+2),straight_line,form)
            except:
                pass
            try:
                acct_worksheet.write('E{}'.format(index*9+simple_index+2),last_month,form)
            except:
                pass
            try:
                acct_worksheet.write('F{}'.format(index*9+simple_index+2),last_month_proj_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('G{}'.format(index*9+simple_index+2),three_month_max_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('H{}'.format(index*9+simple_index+2),six_month_max_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('I{}'.format(index*9+simple_index+2),six_month_performance_diff,percent_fmt)
            except:
                pass

    #         if (three_month_max_diff < -0.3) & (three_month_max_diff != float('-inf')) :
    #             if three_month_dict.get(dp_pub) is None:
    #                 three_month_dict[dp_pub] = {metric:three_month_max_diff}
    #             else:
    #                 three_month_dict[dp_pub][metric] =  three_month_max_diff

    #         if (six_month_max_diff < -0.5) & (six_month_max_diff != float('-inf')) :
    #             if six_month_dict.get(dp_pub) is None:
    #                 six_month_dict[dp_pub] = {metric:six_month_max_diff}
    #             else:
    #                 six_month_dict[dp_pub][metric] =  six_month_max_diff

    #         if (six_month_performance_diff < -0.2) & (six_month_performance_diff != float('-inf')) :
    #             if last_year_month_dict.get(dp_pub) is None:
    #                 last_year_month_dict[dp_pub] = {metric:six_month_performance_diff}
    #             else:
    #                 last_year_month_dict[dp_pub][metric] =  six_month_performance_diff

            if (last_month_proj_diff < 0) & (last_month_proj_diff != float('-inf')) :
                if last_month_dict.get(dp_pub) is None:
                    last_month_dict[dp_pub] = {metric:last_month_proj_diff}
                else:
                    last_month_dict[dp_pub][metric] =  last_month_proj_diff

            if (metric=='Revenue') & (last_month_proj_diff < 0):
                if last_month_proj_diff < -0.2:
                    serious_downtrend_pub[dp_pub] = last_month_proj_diff
                write_last_month = True #note this works because "Revenue" is the first metric
                last_month_count += 1

            if write_last_month:
                try:
                    acct_last_month_sheet.write('C{}'.format((last_month_count)*9+simple_index+2),mtd,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('D{}'.format((last_month_count)*9+simple_index+2),straight_line,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('E{}'.format((last_month_count)*9+simple_index+2),last_month,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('F{}'.format((last_month_count)*9+simple_index+2),last_month_proj_diff,percent_fmt)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('G{}'.format((last_month_count)*9+simple_index+2),three_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('H{}'.format((last_month_count)*9+simple_index+2),six_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('I{}'.format((last_month_count)*9+simple_index+2),six_month_performance_diff,percent_fmt)
                except:
                    pass


    #         #Populate Consecutive DF

    #         decline_series = grouped_consec[metric].diff().lt(0)

    #         zero_series = decline_series.cumsum().where(decline_series, 0) #where no change in series, replace with a 0
    #         true_false_zero_series = zero_series.eq(0)
    #         new_col_name = 'Consecutive Declining {} Months'.format(metric)
    #         grouped_consec[new_col_name] = true_false_zero_series.groupby(true_false_zero_series.cumsum()).cumcount()





        for agg_index,metric in enumerate(agg_metrics):

            # Metric Month to Date
    #         current_month_dp_pub_df = current_month_df[current_month_df['SC_DP&Pub']==dp_pub]
            agg_metric = agg_metrics[metric]
            mtd = current_month_dp_pub_df[agg_metric].agg('sum') / current_month_dp_pub_df['Delivered'].agg('sum')
            if metric == 'Margin':
                mtd = current_month_dp_pub_df[agg_metric].agg('sum') / current_month_dp_pub_df['Revenue'].agg('sum')

            # Straight Line Pacing
    #         if simple_metrics[metric]=='sum':
    #             straight_line = mtd*total_days_in_month/days_passed
    #         else:

            # Last Month's Metric
            last_month = last_month_df[last_month_df['SC_DP&Pub']==dp_pub][agg_metric].sum() / last_month_df[last_month_df['SC_DP&Pub']==dp_pub]['Delivered'].sum()
            if metric == 'Margin':
                last_month =last_month_df[last_month_df['SC_DP&Pub']==dp_pub][agg_metric].sum() / last_month_df[last_month_df['SC_DP&Pub']==dp_pub]['Revenue'].sum()

            if metric=='eCPM':
                mtd *= 1000
                last_month *= 1000
                form = dollar_fmt

                three_month_df_grouped_sc[metric] = 1000*three_month_df_grouped_sc[agg_metrics[metric]]/three_month_df_grouped_sc['Delivered']
                six_month_df_grouped_sc[metric] = 1000*six_month_df_grouped_sc[agg_metrics[metric]]/six_month_df_grouped_sc['Delivered']
                last_month_df_grouped[metric] = 1000*last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Delivered']
                grouped_consec[metric] = 1000*grouped_consec['Revenue']/grouped_consec['Delivered']
            elif metric == 'Margin':
                form = percent_fmt
                three_month_df_grouped_sc[metric] = three_month_df_grouped_sc[agg_metrics[metric]]/three_month_df_grouped_sc['Revenue']
                six_month_df_grouped_sc[metric] = six_month_df_grouped_sc[agg_metrics[metric]]/six_month_df_grouped_sc['Revenue']
                last_month_df_grouped[metric] = last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Revenue']
                grouped_consec[metric] = grouped_consec[agg_metrics[metric]]/grouped_consec['Revenue']
            else:
                form = percent_fmt

                three_month_df_grouped_sc[metric] = three_month_df_grouped_sc[agg_metrics[metric]]/three_month_df_grouped_sc['Delivered']
                six_month_df_grouped_sc[metric] = six_month_df_grouped_sc[agg_metrics[metric]]/six_month_df_grouped_sc['Delivered']
                last_month_df_grouped[metric] = last_month_df_grouped[agg_metrics[metric]]/last_month_df_grouped['Delivered']
                grouped_consec[metric] = grouped_consec[agg_metrics[metric]]/grouped_consec['Delivered']
               


            straight_line = mtd
            # % Difference from 3 Month Max
            three_month_max = three_month_df_grouped_sc[three_month_df_grouped_sc['SC_DP&Pub']==dp_pub][metric].max()
            three_month_max_diff = (straight_line - three_month_max) / three_month_max

    #         % Difference from 12 Month Max
            six_month_max = six_month_df_grouped_sc[six_month_df_grouped_sc['SC_DP&Pub']==dp_pub][metric].max()
            six_month_max_diff = (straight_line - six_month_max) / six_month_max

    #         % Difference from Last Year
            six_month_dp_pub_df = six_month_df[six_month_df['SC_DP&Pub']==dp_pub]
            six_month_performance = six_month_dp_pub_df[agg_metric].sum() / six_month_dp_pub_df['Delivered'].sum()
            if metric == 'Margin':
                six_month_performance = six_month_dp_pub_df[agg_metric].sum() / six_month_dp_pub_df['Revenue'].sum()
            if metric=='eCPM':
                six_month_performance*=1000
            six_month_performance_diff = (straight_line - six_month_performance) / six_month_performance 

            # % Difference from Last Month
            last_month_diff = (straight_line - last_month) / last_month


    #         print(six_month_max_diff==three_month_max_diff)
    #         print(dp_pub,metric,mtd,straight_line,last_month,three_month_max_diff,six_month_max_diff)
            try:
                acct_worksheet.write('C{}'.format(index*9+agg_index+6),mtd,form)
            except:
                pass
            try:
                acct_worksheet.write('D{}'.format(index*9+agg_index+6),straight_line,form)
            except:
                pass
            try:
                acct_worksheet.write('E{}'.format(index*9+agg_index+6),last_month,form)
            except:
                pass
            try:
                acct_worksheet.write('F{}'.format(index*9+agg_index+6),last_month_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('G{}'.format(index*9+agg_index+6),three_month_max_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('H{}'.format(index*9+agg_index+6),six_month_max_diff,percent_fmt)
            except:
                pass
            try:
                acct_worksheet.write('I{}'.format(index*9+agg_index+6),six_month_performance_diff,percent_fmt)
            except:
                pass

    #         if three_month_max_diff < -0.3:
    #             if three_month_dict.get(dp_pub) is None:
    #                 three_month_dict[dp_pub] = {metric:three_month_max_diff}
    #             else:
    #                 three_month_dict[dp_pub][metric] =  three_month_max_diff

    #         if six_month_max_diff < -0.5:
    #             if six_month_dict.get(dp_pub) is None:
    #                 six_month_dict[dp_pub] = {metric:six_month_max_diff}
    #             else:
    #                 six_month_dict[dp_pub][metric] =  six_month_max_diff

    #         if six_month_performance_diff < -0.2:
    #             if last_year_month_dict.get(dp_pub) is None:
    #                 last_year_month_dict[dp_pub] = {metric:six_month_performance_diff}
    #             else:
    #                 last_year_month_dict[dp_pub][metric] =  six_month_performance_diff


            if last_month_diff < -0.2:
                if last_month_dict.get(dp_pub) is None:
                    last_month_dict[dp_pub] = {metric:last_month_diff}
                else:
                    last_month_dict[dp_pub][metric] =  last_month_diff

            if write_last_month:
                try:
                    acct_last_month_sheet.write('C{}'.format((last_month_count)*9+agg_index+6),mtd,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('D{}'.format((last_month_count)*9+agg_index+6),straight_line,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('E{}'.format((last_month_count)*9+agg_index+6),last_month,form)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('F{}'.format((last_month_count)*9+agg_index+6),last_month_diff,percent_fmt)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('G{}'.format((last_month_count)*9+agg_index+6),three_month_max_diff,percent_fmt)
                except:
                    pass
                try:
                    acct_last_month_sheet.write('H{}'.format((last_month_count)*9+agg_index+6),six_month_max_diff,percent_fmt)
                except:
                    pass
                
                try:
                    acct_last_month_sheet.write('I{}'.format((last_month_count)*9+agg_index+6),six_month_performance_diff,percent_fmt)
                except:
                    pass
                
    #         #Populate Consecutive DF
    #         decline_series = grouped_consec[metric].diff().lt(0)

    #         zero_series = decline_series.cumsum().where(decline_series, 0) #where no change in series, replace with a 0
    #         true_false_zero_series = zero_series.eq(0)
    #         new_col_name = 'Consecutive Declining {} Months'.format(metric)
    #         grouped_consec[new_col_name] = true_false_zero_series.groupby(true_false_zero_series.cumsum()).cumcount()
    #         value_before_decline = 'Starting {}'.format(metric)
    # #         date_before_decline = '{} Starting Date'.format(metric)
    #         grouped_consec[value_before_decline] = grouped_consec[new_col_name].apply(lambda x: lexi['Date'].max() + pd.DateOffset(months=-x))



        try: 
            acct_worksheet.merge_range('A{}:A{}'.format(index*9+2,index*9+10), dp_pub, header_fmt)
            acct_worksheet.write('B{}'.format(index*9+2),'Revenue',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+3),'Cost',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+4),'Profit',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+5),'Delivered',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+6),'eCPM',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+7),'CTR',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+8),'Jump Page Click Rate',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+9),'Optout Rate',header_fmt)
            acct_worksheet.write('B{}'.format(index*9+10),'Margin',header_fmt)
        except: 
            pass

        if write_last_month:
            

            acct_last_month_sheet.merge_range('A{}:A{}'.format((last_month_count)*9+2,last_month_count*9+10), dp_pub, header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+2),'Revenue',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+3),'Cost',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+4),'Profit',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+5),'Delivered',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+6),'eCPM',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+7),'CTR',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+8),'Jump Page Click Rate',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+9),'Optout Rate',header_fmt)
            acct_last_month_sheet.write('B{}'.format(last_month_count*9+10),'Margin',header_fmt)



    acct_worksheet.write('A1','SC_DP&Pub',header_fmt)
    acct_worksheet.write('B1','Metric',header_fmt)
    acct_worksheet.write('C1','Month to Date',header_fmt)
    acct_worksheet.write('D1','Straight Line Pacing',header_fmt)
    acct_worksheet.write('E1','Last Month Performance',header_fmt)
    acct_worksheet.write('F1','Percent Difference\nfrom Last Month',header_fmt)
    acct_worksheet.write('G1','Percent Difference\nfrom 3 Month Max',header_fmt)
    acct_worksheet.write('H1','Percent Difference\nfrom 6 Month Max',header_fmt)
    acct_worksheet.write('I1','Percent Difference\nfrom the Month 6 Months ago',header_fmt)

    acct_last_month_sheet.write('A1','SC_DP&Pub',header_fmt)
    acct_last_month_sheet.write('B1','Metric',header_fmt)
    acct_last_month_sheet.write('C1','Month to Date',header_fmt)
    acct_last_month_sheet.write('D1','Straight Line Pacing',header_fmt)
    acct_last_month_sheet.write('E1','Last Month Performance',header_fmt)
    acct_last_month_sheet.write('F1','Percent Difference\nfrom Last Month',header_fmt)
    acct_last_month_sheet.write('G1','Percent Difference\nfrom 3 Month Max',header_fmt)
    acct_last_month_sheet.write('H1','Percent Difference\nfrom 6 MontMax',header_fmt)
    acct_last_month_sheet.write('I1','Percent Difference\nfrom the Month 6 Months ago',header_fmt)


    # three_month_sheet = workbook.add_acct_worksheet('Accounts Declining - 3 Months')
    # three_month_sheet.write('A1','SC_DP&Pub',header_fmt)
    # three_month_sheet.write('B1','Metric',header_fmt)
    # three_month_sheet.write('C1','Projected Percent Difference\nfrom 3 Month Max',header_fmt)
    # three_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (three_month_dict):
    #     for counter,met in enumerate(three_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         three_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         three_month_sheet.write('C{}'.format(row_index+1),three_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         three_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         three_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 


    # six_month_sheet = workbook.add_acct_worksheet('Accounts Declining - 12 Months')
    # six_month_sheet.write('A1','SC_DP&Pub',header_fmt)
    # six_month_sheet.write('B1','Metric',header_fmt)
    # six_month_sheet.write('C1','Projected Percent Difference\nfrom 12 Month Max',header_fmt)
    # six_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (six_month_dict):
    #     for counter,met in enumerate(six_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         six_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         six_month_sheet.write('C{}'.format(row_index+1),six_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         six_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         six_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 

    # last_year_month_sheet = workbook.add_acct_worksheet('Accounts Declining - Last Year')
    # last_year_month_sheet.write('A1','SC_DP&Pub',header_fmt)
    # last_year_month_sheet.write('B1','Metric',header_fmt)
    # last_year_month_sheet.write('C1','Projected Percent Difference\nfrom 12 Month Max',header_fmt)
    # last_year_month_sheet.set_column(0,7,25)
    # row_index = 1
    # for key in (last_year_month_dict):
    #     for counter,met in enumerate(last_year_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         last_year_month_sheet.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         last_year_month_sheet.write('C{}'.format(row_index+1),last_year_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         last_year_month_sheet.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         last_year_month_sheet.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt) 


    # acct_last_month_sheet_any = workbook.add_acct_worksheet('Accounts Declining - Last Month')
    # acct_last_month_sheet_any.write('A1','SC_DP&Pub',header_fmt)
    # acct_last_month_sheet_any.write('B1','Metric',header_fmt)
    # acct_last_month_sheet_any.write('C1','Projected Percent Difference\nfrom Last Month',header_fmt)
    # acct_last_month_sheet_any.set_column(0,7,25)
    # row_index = 1
    # for key in (last_month_dict):
    #     for counter,met in enumerate(last_month_dict[key]):
    #         if met=='Hitpath Offer ID':
    #             met_string='Number of Drops'
    #         else:
    #             met_string=met
    #         acct_last_month_sheet_any.write('B{}'.format(row_index+1),met_string,header_fmt)
    #         acct_last_month_sheet_any.write('C{}'.format(row_index+1),last_month_dict[key][met],percent_fmt)
    #         row_index+=1
    #     if counter==0:
    #         acct_last_month_sheet_any.write('A{}'.format(row_index),key,header_fmt)   
    #     else:
    #         acct_last_month_sheet_any.merge_range('A{}:A{}'.format(row_index-counter,row_index),key,header_fmt)   




    #     acct_worksheet.merge_range('A{}:A{}'.format(index*9+2,index*9+10), dp_pub, header_fmt)

    #     acct_worksheet.write('B{}'.format(index*9+2),'Revenue',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+3),'Delivered',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+4),'Number of Drops',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+5),'Opportunity Cost',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+6),'eCPM Ratio',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+7),'eCPM',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+8),'CTR',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+9),'Jump Page Click Rate',header_fmt)
    #     acct_worksheet.write('B{}'.format(index*9+10),'Optout Rate',header_fmt)



#############################################################################


    # *** Consecutive Decliners Tab ***

    g_consec = lexi.groupby(['DP&Pub',period])
    grouped_consec = g_consec.agg({'Revenue':'sum','Cost':'sum','Profit':'sum','Opportunity Cost':'sum','Delivered':'sum','eCPM Ratio':'mean','Hitpath Offer ID':'count','Jump Page Clicks':'sum','Clicks':'sum','Optout':'sum'})
    grouped_consec = grouped_consec.reset_index()

    grouped_consec['eCPM'] = grouped_consec['Revenue']*1000 / grouped_consec['Delivered']
    grouped_consec['CTR'] = grouped_consec['Clicks'] / grouped_consec['Delivered']
    grouped_consec['Jump Page Click Rate'] = grouped_consec['Jump Page Clicks'] / grouped_consec['Delivered']
    grouped_consec['Optout Rate'] = grouped_consec['Optout'] / grouped_consec['Delivered']
    grouped_consec['Margin'] = grouped_consec['Profit'] / grouped_consec['Revenue']

    #Populate Consecutive DF
    for index,metric in enumerate({**simple_metrics,**agg_metrics}):
        decline_series = grouped_consec.groupby('DP&Pub')[metric].diff().lt(0)

        zero_series = decline_series.cumsum().where(decline_series, 0) #where no change in series, replace with a 0
        true_false_zero_series = zero_series.eq(0)
        new_col_name = 'Consecutive Declining {} Months'.format(metric)
        grouped_consec[new_col_name] = true_false_zero_series.groupby(true_false_zero_series.cumsum()).cumcount()
        value_before_decline = 'Starting {}'.format(metric)
        date_before_decline = 'Date Starting {}'.format(metric)

        grouped_consec[date_before_decline] = grouped_consec[new_col_name].apply(lambda x: lexi['Date'].max() + pd.DateOffset(months=-x)).dt.to_period("M")


        def find_start_value(date,pub):
            return_df = grouped_consec[(grouped_consec['Date']==date) & (grouped_consec['DP&Pub']==pub)][metric]
            return return_df.values[0] if len(return_df)>0 else 0

        grouped_consec[value_before_decline] = grouped_consec.apply(lambda x: find_start_value(x[date_before_decline], x['DP&Pub']), axis=1)

        previous_date = (lexi['Date'].max() + pd.DateOffset(months=-1)).to_period("M")
        grouped_consec['Previous {}'.format(metric)] = grouped_consec['DP&Pub'].apply(lambda x: grouped_consec[(grouped_consec['Date']==previous_date) & (grouped_consec['DP&Pub']==x)][metric].values[0] if len(grouped_consec[(grouped_consec['Date']==previous_date) & (grouped_consec['DP&Pub']==x)][metric])>0 else 0 )    


    consec_cols = [col for col in grouped_consec.columns if 'Consecutive' in col]
    consecutive_df = grouped_consec[(grouped_consec[consec_cols].gt(2).any(axis=1)) & (grouped_consec['Date'].dt.month==current_month)
            & (grouped_consec['Date'].dt.year==current_year)]


    # workbook = xlsxwriter.Workbook('Account Feedback - {}.xlsx'.format(date.today().strftime("%m_%d_%Y")))
    consecutive_sheet = workbook.add_worksheet('Consecutive Decliners')
    # percent_fmt = workbook.add_format({'num_format': '0.00%'})
    # dollar_fmt = workbook.add_format({'num_format': '$#,###.##'})
    # no_fmt = workbook.add_format({'num_format': '#,###.##'})

    row_counter = 1
    for pub_index, pub in enumerate(consecutive_df['DP&Pub'].unique()):
        pub_counter = row_counter+1
        pub_df = consecutive_df[consecutive_df['DP&Pub']==pub]
        metric_indices = list(np.where((pub_df[consec_cols]>=2).T==True)[0])
        metrics_to_write = [consec_cols[x].split('Consecutive Declining ')[1].split(' Months')[0] for x in metric_indices]
        for index, metric in enumerate(metrics_to_write):
            row_counter+=1

            if (metric=='Revenue') | (metric=='Opportunity Cost') |(metric=='Profit') | (metric=='Cost') | (metric=='eCPM')   :
                form = dollar_fmt
            elif metric=='Delivered':
                form = no_fmt
            elif metric=='eCPM Ratio':
                form = workbook.add_format({'num_format': '0.00'})
            else:
                form = percent_fmt

            if metric=='Hitpath Offer ID':
                consecutive_sheet.write('B{}'.format(row_counter),'Number of Drops',header_fmt)
            else:
                consecutive_sheet.write('B{}'.format(row_counter),metric,header_fmt)

            months_in_decline = pub_df['Consecutive Declining {} Months'.format(metric)].values[0]
            consecutive_sheet.write('C{}'.format(row_counter),months_in_decline,no_fmt)

            mtd = pub_df[(metric)].values[0]
            try:
                consecutive_sheet.write('D{}'.format(row_counter),mtd,form)
            except:
                pass

            if (metric in simple_metrics): 
                if (simple_metrics[metric]=='sum') | (simple_metrics[metric]=='count'):
                    straight_line = mtd*total_days_in_month/days_passed
                else:
                    straight_line = mtd
            else:
                    straight_line = mtd
            start_value = pub_df['Starting {}'.format(metric)].values[0]
            percent_from_start = (straight_line - start_value)/start_value

            try:
                consecutive_sheet.write('E{}'.format(row_counter),straight_line,form)
            except:
                pass

            try:
                consecutive_sheet.write('F{}'.format(row_counter),percent_from_start,percent_fmt)
            except:
                pass

            try:
                consecutive_sheet.write('G{}'.format(row_counter),start_value,form)
            except:
                pass

            last_month_performance = pub_df['Previous {}'.format(metric)].values[0]
            percent_from_last_month = (straight_line - last_month_performance) / last_month_performance

            try:
                consecutive_sheet.write('H{}'.format(row_counter),percent_from_last_month,percent_fmt)
            except:
                pass
            try:
                consecutive_sheet.write('I{}'.format(row_counter),last_month_performance,form)
            except:
                pass


        if pub_counter==row_counter:
            consecutive_sheet.write('A{}'.format(row_counter),pub,header_fmt)
        else:
            consecutive_sheet.merge_range('A{}:A{}'.format(pub_counter,row_counter), pub, header_fmt)


    header_fmt = workbook.add_format({'bold': 1,'align': 'center','valign': 'vcenter'})
    consecutive_sheet.write('A1','DP&Pub',header_fmt)
    consecutive_sheet.write('B1','Metric',header_fmt)
    consecutive_sheet.write('C1','Number of Months in Decline',header_fmt)
    consecutive_sheet.write('D1','Month to Date',header_fmt)
    consecutive_sheet.write('E1','Straight Line Projection',header_fmt)
    consecutive_sheet.write('F1','Percent Difference\nfrom Start of Decline',header_fmt)
    consecutive_sheet.write('G1','Value at Start of Decline',header_fmt)
    consecutive_sheet.write('H1','Percent Difference\nfrom Last Month',header_fmt)
    consecutive_sheet.write('I1','Value from Last Month',header_fmt)
    consecutive_sheet.set_column(0,8,25)
    workbook.close()

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:84: RuntimeWarning: invalid value encountered in scalar divide
  last_7d_diff_subscribed = (today_subscribed - last_7d_subscribed) / last_7d_subscribed
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:71: RuntimeWarning: invalid value encountered in scalar divide
  subscribed_pct = today_subscribed / today_post
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:83: RuntimeWarning: invalid value encountered in scalar divide
  last_7d_diff = (today_post - last_7d_post) / last_7d_post
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:168: RuntimeWarning: invalid value encountered in scalar divide
  last_7d_diff = (today_segment - last_7d_segment) / last_7d_segment
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:158: RuntimeWarning: invalid value encountered in scalar divide
  last_7d_segment =  last_7d_sc_dp_pub_df['count'].sum() / len(last_7d_sc_dp_pub_df['Date']

D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:1029: RuntimeWarning: invalid value encountered in scalar divide
  three_month_max_diff = (straight_line - three_month_max) / three_month_max
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:1033: RuntimeWarning: invalid value encountered in scalar divide
  six_month_max_diff = (straight_line - six_month_max) / six_month_max
D:\Users\lilig\AppData\Local\Temp\1\ipykernel_2472\2434250363.py:999: RuntimeWarning: invalid value encountered in scalar divide
  last_month =last_month_df[last_month_df['SC_DP&Pub']==dp_pub][agg_metric].sum() / last_month_df[last_month_df['SC_DP&Pub']==dp_pub]['Revenue'].sum()


In [5]:
 
os.chdir( filepath.output_folder) 
filename = 'Account Feedback - {}.xlsx'.format(date.today().strftime("%m_%d_%Y"))
toaddr = ['tinamarie@rxmg.com','r.woodward@rxmg.com','g.chao@rxmg.com','nathan@rxmg.com','nikki@rxmg.com','n.baker@rxmg.com','lili@rxmg.com','nina@rxmg.com']
#toaddr1 = ['nikki@rxmg.com','n.baker@rxmg.com','lili@rxmg.com']
#toaddr = ['lili@rxmg.com']
today = date.today().strftime("%m_%d_%Y")
subject_line = f"SMS Team Daily Alert Report - {today}"
email_body = "Hi Team,\n\n"
#email_body += zero_ot_email_body +  '\n\n\n'
email_body += delivered_email_body +  '\n\n\n'
email_body += push_email_body +  '\n\n\n'
email_body += optout_alert_email_body + '\n\n\n'
email_body += post_downtrend_email_body + '\n\n\n'
email_body += post_uptrend_email_body + '\n\n\n'
email_body += post_alert_email_body +  '\n\n\n'
email_body += subscribed_email_body +  '\n\n\n'
email_body += subcribed_uptrend_email_body +  '\n\n\n'
email_body += subcribed_downtrend_email_body +  '\n\n\n'
email_body += segment_email_body +  '\n\n\n'
email_body +="* Here're Downtrending Account: Projected Monthly Revenue Sees Over 20% Decrease from Last Month \n\n"
for key in serious_downtrend_pub:
    email_body += f"\t{key}: Revenue for this month is projected to be down by {round(serious_downtrend_pub[key]*100, 2)}% from last month.\n"

email_body += '\n\nHave a nice day!\n\nThanks\nLili Guo\n\n'

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)
#for i in toaddr1:
    #send_email.send_email([filename],subject_line,email_body,i)
print(toaddr)


connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp'
connect: b'smtp.gmail.com ESMTP d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [52.38.54.186]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your 

reply: b'250 2.0.0 OK  1710457558 d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457558 d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp'
data: (250, b'2.0.0 OK  1710457558 d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection d2-20020a05620a158200b00789cd996cbcsm1334080qkk.134 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp'
connect: b'smtp.gmail.com ESMTP kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-S

reply: b'250 2.0.0 OK  1710457560 kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457560 kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp'
data: (250, b'2.0.0 OK  1710457560 kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection kd13-20020a056214400d00b0068ff79d8d97sm1081194qvb.41 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp'
connect: b'smtp.gmail.com ESMTP b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250

reply: b'250 2.0.0 OK  1710457564 b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457564 b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp'
data: (250, b'2.0.0 OK  1710457564 b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection b13-20020ac8678d000000b0042f0008b8b7sm1294032qtp.35 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp'
connect: b'smtp.gmail.com ESMTP g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 35

reply: b'250 2.0.0 OK  1710457566 g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457566 g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp'
data: (250, b'2.0.0 OK  1710457566 g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection g5-20020ac84b65000000b0042f3882cd05sm1296044qts.42 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp'
connect: b'smtp.gmail.com ESMTP c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 3588

reply: b'250 2.0.0 OK  1710457569 c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457569 c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp'
data: (250, b'2.0.0 OK  1710457569 c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection c20-20020ac853d4000000b0042f4b6ff409sm1292877qtq.52 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp'
connect: b'smtp.gmail.com ESMTP fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-S

reply: b'250 2.0.0 OK  1710457571 fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457571 fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp'
data: (250, b'2.0.0 OK  1710457571 fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection fq9-20020a056214258900b00690d5310c55sm1068351qvb.114 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp'
connect: b'smtp.gmail.com ESMTP g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 

reply: b'250 2.0.0 OK  1710457573 g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457573 g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp'
data: (250, b'2.0.0 OK  1710457573 g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection g4-20020a0cf844000000b006915face0easm890565qvo.70 - gsmtp'
connect: to ('smtp.gmail.com', 587) None
reply: b'220 smtp.gmail.com ESMTP h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp\r\n'
reply: retcode (220); Msg: b'smtp.gmail.com ESMTP h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp'
connect: b'smtp.gmail.com ESMTP h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp'
send: 'ehlo [100.96.2.115]\r\n'
reply: b'250-smtp.gmail.com at your service, [52.38.54.186]\r\n'
reply: b'250-SIZE 358825

['tinamarie@rxmg.com', 'r.woodward@rxmg.com', 'g.chao@rxmg.com', 'nathan@rxmg.com', 'nikki@rxmg.com', 'n.baker@rxmg.com', 'lili@rxmg.com', 'nina@rxmg.com']


reply: b'250 2.0.0 OK  1710457575 h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1710457575 h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp'
data: (250, b'2.0.0 OK  1710457575 h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection h15-20020a05620a10af00b0078861322748sm1355867qkk.102 - gsmtp'
